In [1]:
from django_for_jupyter import init_django

init_django()

/home/javier


In [2]:
from precios.models import (Site, SiteURLResults, Marcas, Articulos, Vendedores, Unifica)
import re

In [24]:
from collections import defaultdict

# ID del supermercado para el cual deseas generar el informe
id_supermercado = 9  # Ingresa aquí el ID del supermercado deseado era 54

try:
    # Intenta obtener el supermercado con el ID proporcionado
    supermercado = Site.objects.get(id=id_supermercado)
except Site.DoesNotExist:
    print("Supermercado no encontrado.")
else:
    # Filtra los vendedores para el sitio del supermercado
    vendedores_supermercado = Vendedores.objects.filter(vendidoen__site=supermercado)
    # vendedores_supermercado = Vendedores.objects.filter(articulo__marca__id=id_marca)

    # Un diccionario para almacenar los resultados del informe
    informe = defaultdict(int)

    # Itera a través de los vendedores del supermercado
    tot_articulos = 0
    no_encontrados = 0
    for vendedor in vendedores_supermercado:
        articulo = vendedor.articulo
        try:
            vendidoen = Vendedores.objects.filter(vendidoen__site=supermercado, articulo=articulo).values_list('vendidoen',flat=True).first()
            # for vendidoen in vendidosen:
            tot_articulos = tot_articulos  + 1
            surlres = SiteURLResults.objects.get(id=vendidoen)
            posicion = surlres.posicion
                

            # print(f'{articulo}-Nombre={surlres.nombre} artref={surlres.num_articulos}, posicion={surlres.posicion}')
            informe[surlres.num_articulos, posicion] += 1

        except Exception as e:
            no_encontrados = no_encontrados + 1
            print(f"Error : {e} articulo={articulo}")
            pass    

    # Ordenar los resultados del informe por el segundo valor del puesto
    informe_ordenado = dict(sorted(informe.items(), key=lambda item: item[0][1]))

    # Imprime los resultados del informe ordenados
    print(f"Informe para el supermercado {id_supermercado} {supermercado.siteName} con {tot_articulos}:")
    suma_cantidad = 0
    suma_por_puesto = defaultdict(int)
    inicio = True
    saldo = tot_articulos
    pp0 = 0
    pp1 = 0
    for puesto, cantidad in informe_ordenado.items():
        if inicio:
            inicio = False
            pp0 = puesto[0]
            pp1 = puesto[1]

        if pp1 != puesto[1]:
            print(f'suma_por_puesto {pp1}={suma_por_puesto[pp1]} {round(suma_por_puesto[pp1] / saldo, 2 ) * 100}% ')
            pp0 = puesto[0]
            pp1 = puesto[1]

        
        suma_cantidad += cantidad
        if puesto[0] == 1 and puesto[1] == 1:
            inicial = cantidad
            print(f"Inicial: {inicial} productos {round((inicial / saldo ),2) * 100}%  ")
            saldo = saldo - inicial
            print(f'Saldo={saldo}')
            print()
            
        else:
            # print(f"Puesto {puesto}: {cantidad} productos ")
            suma_por_puesto[puesto[1]] += cantidad

    print(f'suma_por_puesto {pp1}={suma_por_puesto[pp1]} {round(suma_por_puesto[pp1] / saldo, 2 ) * 100}%')

    print('Suma', suma_cantidad, ' no_encontrados=', no_encontrados)
    


Informe para el supermercado 9 Cugat con 2644:
Inicial: 1673 productos 63.0%  
Saldo=971

suma_por_puesto 1=332 34.0% 
suma_por_puesto 2=342 35.0% 
suma_por_puesto 3=141 15.0% 
suma_por_puesto 4=72 7.000000000000001% 
suma_por_puesto 5=40 4.0% 
suma_por_puesto 6=22 2.0% 
suma_por_puesto 7=9 1.0% 
suma_por_puesto 8=8 1.0% 
suma_por_puesto 9=4 0.0% 
suma_por_puesto 12=1 0.0%
Suma 2644  no_encontrados= 0


In [31]:
from collections import defaultdict
import pandas as pd


def generar_informe_supermercados():
    # Obtener la lista de todos los supermercados (puedes ajustar esto según tu modelo)
    lista_supermercados = Site.objects.filter(enable=True)[2:10]

    # Un diccionario para almacenar los resultados del informe por supermercado y posición
    informe_supermercados = defaultdict(lambda: defaultdict(int))

    for supermercado in lista_supermercados:
        vendedores_supermercado = Vendedores.objects.filter(vendidoen__site=supermercado)

        informe = defaultdict(int)
        tot_articulos = 0
        no_encontrados = 0

        for vendedor in vendedores_supermercado:
            articulo = vendedor.articulo
            try:
                vendidoen = Vendedores.objects.filter(vendidoen__site=supermercado, articulo=articulo).values_list('vendidoen', flat=True).first()
                tot_articulos = tot_articulos + 1
                surlres = SiteURLResults.objects.get(id=vendidoen)
                posicion = surlres.posicion

                informe[surlres.num_articulos, posicion] += 1

            except Exception as e:
                no_encontrados = no_encontrados + 1
                pass

        informe_ordenado = dict(sorted(informe.items(), key=lambda item: item[0][1]))

        suma_cantidad = 0
        suma_por_puesto = defaultdict(int)
        saldo = tot_articulos
        pp0 = 0
        pp1 = 0

        for puesto, cantidad in informe_ordenado.items():
            if pp0 != puesto[0]:
                pp0 = puesto[0]
                pp1 = puesto[1]
                saldo = tot_articulos

            suma_cantidad += cantidad
            suma_por_puesto[puesto[1]] += cantidad

            if puesto[0] == 1 and puesto[1] == 1:
                inicial = cantidad
                saldo = saldo - inicial + 1

        for puesto in sorted(set(puesto[1] for puesto in informe_ordenado)):
            informe_supermercados[puesto][supermercado.siteName] = round(suma_por_puesto[puesto] / saldo, 2) * 100

    # Imprimir el informe
    print("Posición\t" + "\t".join(site.siteName for site in lista_supermercados))
    for puesto, porcentajes in informe_supermercados.items():
        porcentajes_str = "\t".join(f"{porcentaje}%" for porcentaje in porcentajes.values())
        print(f"{puesto}\t{porcentajes_str}")

    # Crear un DataFrame de pandas a partir del diccionario informe_supermercados
    df = pd.DataFrame.from_dict(informe_supermercados, orient='index')

    # Imprimir el DataFrame
    print(df)
# Llamada a la función para generar el informe
generar_informe_supermercados()


Posición	Santa Isabel	Cugat	Sotocopias	Supermercados Eltit	El Cielo	Alvi	Wild Fit	DeliMarket
1	75.0%	76.0%	17800.0%	83.0%	74.0%	67.0%	100.0%	62.0%
2	17.0%	13.0%	9.0%	12.0%	18.0%	17.0%
3	4.0%	5.0%	4.0%	4.0%	7.000000000000001%	6.0%
4	2.0%	3.0%	2.0%	4.0%	5.0%	7.000000000000001%
5	1.0%	2.0%	1.0%	3.0%	2.0%	4.0%
6	0.0%	1.0%	0.0%	1.0%	1.0%	2.0%
7	0.0%	0.0%	0.0%	1.0%	0.0%	2.0%
8	0.0%	0.0%	0.0%	0.0%	0.0%	0.0%
9	0.0%	0.0%	0.0%	0.0%
10	0.0%	0.0%
12	0.0%	0.0%
13	0.0%


In [33]:

from collections import defaultdict

def generar_informe_supermercados():
    # Obtener la lista de todos los supermercados (puedes ajustar esto según tu modelo)
    lista_supermercados = Site.objects.filter(enable=True)[2:10]

    # Un diccionario para almacenar los resultados del informe por supermercado y posición
    informe_supermercados = defaultdict(lambda: defaultdict(int))

    for supermercado in lista_supermercados:
        vendedores_supermercado = Vendedores.objects.filter(vendidoen__site=supermercado)

        informe = defaultdict(int)
        tot_articulos = 0
        no_encontrados = 0

        for vendedor in vendedores_supermercado:
            articulo = vendedor.articulo
            try:
                vendidoen = Vendedores.objects.filter(vendidoen__site=supermercado, articulo=articulo).values_list('vendidoen', flat=True).first()
                tot_articulos = tot_articulos + 1
                surlres = SiteURLResults.objects.get(id=vendidoen)
                posicion = surlres.posicion

                informe[surlres.num_articulos, posicion] += 1

            except Exception as e:
                no_encontrados = no_encontrados + 1
                pass

        informe_ordenado = dict(sorted(informe.items(), key=lambda item: item[0][1]))

        suma_cantidad = 0
        suma_por_puesto = defaultdict(int)
        saldo = tot_articulos
        pp0 = 0
        pp1 = 0

        for puesto, cantidad in informe_ordenado.items():
            if pp0 != puesto[0]:
                pp0 = puesto[0]
                pp1 = puesto[1]
                saldo = tot_articulos

            suma_cantidad += cantidad
            suma_por_puesto[puesto[1]] += cantidad

            if puesto[0] == 1 and puesto[1] == 1:
                inicial = cantidad
                saldo = saldo - inicial + 1

        for puesto in sorted(set(puesto[1] for puesto in informe_ordenado)):
            porcentajes = {site.siteName: round(suma_por_puesto[puesto] / saldo * 100, 2) for site in lista_supermercados}
            informe_supermercados[puesto] = porcentajes

    # Imprimir el informe
    print("Posición\t" + "\t".join(site.siteName for site in lista_supermercados))
    for puesto, porcentajes in informe_supermercados.items():
        porcentajes_str = "\t".join(f"{porcentaje}%" for porcentaje in porcentajes.values())
        print(f"{puesto}\t{porcentajes_str}")

# Llamada a la función para generar el informe
generar_informe_supermercados()


Posición	Santa Isabel	Cugat	Sotocopias	Supermercados Eltit	El Cielo	Alvi	Wild Fit	DeliMarket
1	62.26%	62.26%	62.26%	62.26%	62.26%	62.26%	62.26%	62.26%
2	16.98%	16.98%	16.98%	16.98%	16.98%	16.98%	16.98%	16.98%
3	5.66%	5.66%	5.66%	5.66%	5.66%	5.66%	5.66%	5.66%
4	6.6%	6.6%	6.6%	6.6%	6.6%	6.6%	6.6%	6.6%
5	3.77%	3.77%	3.77%	3.77%	3.77%	3.77%	3.77%	3.77%
6	2.36%	2.36%	2.36%	2.36%	2.36%	2.36%	2.36%	2.36%
7	1.89%	1.89%	1.89%	1.89%	1.89%	1.89%	1.89%	1.89%
8	0.47%	0.47%	0.47%	0.47%	0.47%	0.47%	0.47%	0.47%
9	0.14%	0.14%	0.14%	0.14%	0.14%	0.14%	0.14%	0.14%
10	0.07%	0.07%	0.07%	0.07%	0.07%	0.07%	0.07%	0.07%
12	0.02%	0.02%	0.02%	0.02%	0.02%	0.02%	0.02%	0.02%
13	0.02%	0.02%	0.02%	0.02%	0.02%	0.02%	0.02%	0.02%


In [37]:
import pandas as pd
from collections import defaultdict

# Obtener la lista de todos los supermercados (puedes ajustar esto según tu modelo)
lista_supermercados = Site.objects.filter(enable=True)[2:3]

# Crear un diccionario para almacenar los resultados del informe por supermercado y posición
informe_supermercados = defaultdict(lambda: defaultdict(int))

for supermercado in lista_supermercados:
    vendedores_supermercado = Vendedores.objects.filter(vendidoen__site=supermercado)

    informe = defaultdict(int)
    tot_articulos = 0
    no_encontrados = 0

    for vendedor in vendedores_supermercado:
        articulo = vendedor.articulo
        try:
            vendidoen = Vendedores.objects.filter(vendidoen__site=supermercado, articulo=articulo).values_list('vendidoen', flat=True).first()
            tot_articulos = tot_articulos + 1
            surlres = SiteURLResults.objects.get(id=vendidoen)
            posicion = surlres.posicion

            informe[surlres.num_articulos, posicion] += 1

        except Exception as e:
            no_encontrados = no_encontrados + 1
            pass

    informe_ordenado = dict(sorted(informe.items(), key=lambda item: item[0][1]))

    suma_cantidad = 0
    suma_por_puesto = defaultdict(int)
    saldo = tot_articulos
    pp0 = 0
    pp1 = 0

    for puesto, cantidad in informe_ordenado.items():
        if pp0 != puesto[0]:
            pp0 = puesto[0]
            pp1 = puesto[1]
            saldo = tot_articulos

        suma_cantidad += cantidad
        suma_por_puesto[puesto[1]] += cantidad

        if puesto[0] == 1 and puesto[1] == 1:
            inicial = cantidad
            saldo = saldo - inicial + 1

    for puesto in sorted(set(puesto[1] for puesto in informe_ordenado)):
        porcentaje = round(suma_por_puesto[puesto] / saldo * 100, 2)
        informe_supermercados[puesto][supermercado.siteName] = porcentaje

# Crear un DataFrame de pandas a partir del diccionario informe_supermercados
df = pd.DataFrame.from_dict(informe_supermercados, orient='index')

# Imprimir el DataFrame
print(df)


OperationalError: (2013, 'Lost connection to MySQL server during query')